In [4]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=9b1f44eae4be871238f67ab6287d0b43f684d24b98e04190f6db2abbc5e910ac
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pickle

tqdm.pandas()

In [56]:
path = '/content/drive/MyDrive/Магистратура/Экспертные и рекомендательные системы/Курсовая работа v2/data/'

interactions_df = pd.read_csv(path+'interactions_clean.csv')
items_df = pd.read_csv(path+'items_clean.csv')
users_df = pd.read_csv(path+'users_clean.csv')

interactions_df = interactions_df[interactions_df['user_id'].isin(users_df['user_id'].unique())]
interactions_df = interactions_df[interactions_df['item_id'].isin(items_df['id'].unique())]

interactions_df.shape, items_df.shape, users_df.shape

((1400003, 7), (59599, 5), (142888, 3))

In [57]:
interactions_df.columns

Index(['user_id', 'item_id', 'rating', 'start_date', 'year', 'month',
       'sequence'],
      dtype='object')

In [3]:
def create_items_feature(genres, authors, year):
  try:
    concat = genres + ',' + authors + ',' + str(year)
  except:
    print(genres, authors, year)
  return concat


def create_users_feature(age, sex):
  concat = age + ',' + sex
  return concat

In [4]:
items_df['features'] = items_df.iloc[:, 2:].progress_apply(lambda row: create_items_feature(*row), axis=1)
users_df['features'] = users_df.iloc[:, 1:].progress_apply(lambda row: create_users_feature(*row), axis=1)

  0%|          | 0/59599 [00:00<?, ?it/s]

  0%|          | 0/142888 [00:00<?, ?it/s]

In [5]:
items_df = items_df[['id', 'features']]
users_df = users_df[['user_id', 'features']]

# Daatset

In [6]:
from lightfm.data import Dataset
from lightfm import LightFM
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
def custom_tokenizer(text):
    text = text.strip().split(',')
    text = [word.strip() for word in text]
    return text

In [27]:
# users_df = users_df[users_df['user_id'].isin(interactions_df['user_id'].unique())]
# items_df = items_df[items_df['id'].isin(interactions_df['item_id'].unique())]

In [8]:
vectorizer_users = CountVectorizer(max_features=20000, lowercase=False, tokenizer=custom_tokenizer) # max_features=20000, lowercase=False, tokenizer=custom_tokenizer
vectorizer_items = CountVectorizer(max_features=50000, lowercase=False, tokenizer=custom_tokenizer) # max_features=50000, lowercase=False, tokenizer=custom_tokenizer

In [9]:
users_features = vectorizer_users.fit_transform(users_df['features'])
items_features = vectorizer_items.fit_transform(items_df['features'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
users_features_names = vectorizer_users.get_feature_names_out()
items_features_names = vectorizer_items.get_feature_names_out()

In [31]:
# with open(path+'countvectorizer_users.pickle', 'wb') as handle:
#   pickle.dump(vectorizer_users, handle)

# with open(path+'countvectorizer_items.pickle', 'wb') as handle:
#   pickle.dump(vectorizer_items, handle)

In [11]:
dataset = Dataset()

In [12]:
user_ids_buffered = [x for x in interactions_df['user_id'].unique()]
item_ids_buffered = [x for x in interactions_df['item_id'].unique()]

user_ids_buffered = np.array(user_ids_buffered)
item_ids_buffered = np.array(item_ids_buffered)

In [13]:
dataset.fit(
    users=user_ids_buffered,
    items=item_ids_buffered,
    item_features=items_features_names,
    user_features=users_features_names
)

In [36]:
(interactions, weights) = dataset.build_interactions(((k, v, w)
                                                       for k, v, w in
                                                       zip(list(interactions_df['user_id'].values),
                                                           list(interactions_df['item_id'].values),
                                                           list(interactions_df['rating'].values+1))))

In [42]:
model = LightFM(loss='warp', random_state=42, item_alpha=0.00001, user_alpha=0.00001, no_components=32)
# model = LightFM(no_components=200, loss='warp', learning_schedule='adagrad', user_alpha=6e-5, item_alpha=2e-5, learning_rate=0.01, max_sampled=150, random_state=42)

model.fit(
    interactions=interactions,
    sample_weight=weights,
    item_features=items_features,
    user_features=users_features,
    verbose=True,
    epochs=10,
    num_threads=20,
)

Epoch: 100%|██████████| 10/10 [01:08<00:00,  6.87s/it]


In [49]:
with open(path+'lightfm_model.pickle', 'wb') as handle:
  pickle.dump(model, handle)

# Cross Validation

In [18]:
import itertools
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k

In [ ]:
train, test = random_train_test_split(interactions, test_percentage=0.1, random_state=42)
train_weights, test_weights = random_train_test_split(weights, test_percentage=0.1, random_state=42)

In [23]:
def sample_hyperparameters():
    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["warp"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            # "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 20),
            "random_state": [42]
        }


def random_search(train, test, num_samples=10):

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams)
        model.fit(
            interactions=train,
            sample_weight=train_weights,
            item_features=items_features,
            user_features=users_features,
            verbose=True,
            epochs=num_epochs,
            num_threads=20,
          )

        auc_score_train = auc_score(model, train, user_features=users_features, item_features=items_features, train_interactions=train, num_threads=20, check_intersections=False).mean()
        auc_score_test = auc_score(model, test, user_features=users_features, item_features=items_features, train_interactions=train, num_threads=20, check_intersections=False).mean()

        train_precision = precision_at_k(model, train, user_features=users_features, item_features=items_features, k=5, num_threads=20, check_intersections=False).mean()
        test_precision = precision_at_k(model, test, user_features=users_features, item_features=items_features, k=5, num_threads=20, check_intersections=False).mean()

        hyperparams["num_epochs"] = num_epochs

        print(auc_score_train, auc_score_test, train_precision, test_precision, hyperparams, model)

        yield (auc_score_train, auc_score_test, train_precision, test_precision, hyperparams, model)

In [ ]:
(auc_score_train, auc_score_test, train_precision, test_precision, hyperparams, model) = max(random_search(train, test), key=lambda x: x[1])

Epoch: 100%|██████████| 13/13 [00:53<00:00,  4.11s/it]


0.6086827 {'no_components': 39, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.03028263129356733, 'item_alpha': 2.2828269553498666e-09, 'user_alpha': 8.04761306653879e-09, 'random_state': [42], 'num_epochs': 13} <lightfm.lightfm.LightFM object at 0x780e3c687850>


Epoch: 100%|██████████| 11/11 [00:41<00:00,  3.79s/it]


0.58271784 {'no_components': 23, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.12323427680962146, 'item_alpha': 1.9688329121251424e-09, 'user_alpha': 5.124845095139062e-09, 'random_state': [42], 'num_epochs': 11} <lightfm.lightfm.LightFM object at 0x780e3c6861a0>


Epoch: 100%|██████████| 5/5 [00:26<00:00,  5.24s/it]


0.58323276 {'no_components': 36, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.016718167960778695, 'item_alpha': 2.967275170035453e-09, 'user_alpha': 4.716394875416876e-09, 'random_state': [42], 'num_epochs': 5} <lightfm.lightfm.LightFM object at 0x780e3c6864a0>


Epoch: 100%|██████████| 17/17 [01:38<00:00,  5.78s/it]


0.5744952 {'no_components': 40, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.020330631481341167, 'item_alpha': 3.4564630383934545e-08, 'user_alpha': 1.5552924771535074e-08, 'random_state': [42], 'num_epochs': 17} <lightfm.lightfm.LightFM object at 0x780e3c6854b0>


Epoch: 100%|██████████| 9/9 [00:23<00:00,  2.66s/it]


# Recommendations

In [20]:
items_df_first = pd.read_csv(path+'items_clean.csv')

In [47]:
user_id = user_ids_buffered[111]
scores = model.predict(dataset.mapping()[0][user_id],
                       np.arange(dataset.interactions_shape()[1]),
                       user_features=users_features,
                       item_features=items_features)

In [48]:
recommended_inds = item_ids_buffered[np.argsort(-scores)][:10]
already_rated_inds = interactions_df[interactions_df['user_id'] == user_id]['item_id'].values

# print('Прочтенные')
# print()
# display(items_df_first[items_df_first['id'].isin(already_rated_inds)])
print()
print('Рекомендованные')
display(items_df_first[items_df_first['id'].isin(recommended_inds)])


Рекомендованные


,id,title,genres,authors,year
735,65376,Калямбра,Современная русская литература,Александр Покровский,2008
6608,63990,Тринадцатая сказка,Современная зарубежная литература,Диана Сеттерфилд,2013
8218,155451,Выйди из зоны комфорта. Измени свою жизнь,"Личная эффективность,Саморазвитие / личностный...",Брайан Трейси,2013
19612,109201,Яблоки из сада Шлицбутера,Современная русская литература,Дина Рубина,year_unknown
28950,195516,Приключения Озорули и Хитрули. Сказки-обучалки,Книги для детей,Владислав Щербак,year_unknown
29369,216337,Русские женщины,"Старинная литература,Русская классика",Николай Некрасов,year_unknown
33326,316995,Самая таинственная тайна и другие сюжеты,"Общая история,Публицистика",Борис Акунин,2014
36380,160544,Легенды и мифы о Сталине,"Биографии и мемуары,Документальная литература",Владимир Суходеев,2009
44282,288531,Инферно,"Триллеры,Зарубежная фантастика,Детективная фан...",Дэн Браун,2013
58944,298161,Сборник задач по дисциплине «Инвестиции»,Ценные бумаги / инвестиции,Руслан Бодров,2017


In [46]:
recommended_inds = item_ids_buffered[np.argsort(-scores)][:10]
already_rated_inds = interactions_df[interactions_df['user_id'] == user_id]['item_id'].values

# print('Прочтенные')
# print()
# display(items_df_first[items_df_first['id'].isin(already_rated_inds)])
print()
print('Рекомендованные')
display(items_df_first[items_df_first['id'].isin(recommended_inds)])


Рекомендованные


,id,title,genres,authors,year
735,65376,Калямбра,Современная русская литература,Александр Покровский,2008
15552,232537,"Поступай как женщина, думай как мужчина. Почем...","Семейная психология,Практическая психология,За...",Стив Харви,2009
15715,276195,Брат за брата,Криминальные боевики,Владимир Колычев,2001
21704,264997,После,"Зарубежные любовные романы,Современные любовны...",Анна Тодд,2014
28950,195516,Приключения Озорули и Хитрули. Сказки-обучалки,Книги для детей,Владислав Щербак,year_unknown
33326,316995,Самая таинственная тайна и другие сюжеты,"Общая история,Публицистика",Борис Акунин,2014
34259,53518,Sapiens. Краткая история человечества,"Научно-популярная литература,Зарубежная образо...",Юваль Ной Харари,2011
36380,160544,Легенды и мифы о Сталине,"Биографии и мемуары,Документальная литература",Владимир Суходеев,2009
38532,176898,Седмица Трехглазого,Исторические детективы,Борис Акунин,2017
44282,288531,Инферно,"Триллеры,Зарубежная фантастика,Детективная фан...",Дэн Браун,2013


In [27]:
items_df_first = pd.read_csv(path+'items_clean.csv')
users_df_first = pd.read_csv(path+'users_clean.csv')

In [29]:
items_df_first

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков Щедрин,1873
...,...,...,...,...,...
59594,45640,МК Московский Комсомолец 291-2019,"Политология,Книги по экономике,Газеты",Authors_Unknown,2019
59595,321616,МК Московский Комсомолец 292-2019,"Политология,Книги по экономике,Газеты",Authors_Unknown,2019
59596,125582,Известия 248-249-2019,"Политология,Общая история,Газеты",Authors_Unknown,2019
59597,33188,Men's Health 01-2020,Журнальные издания,Authors_Unknown,2019


In [67]:
items_df_first

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков Щедрин,1873
...,...,...,...,...,...
59594,45640,МК Московский Комсомолец 291-2019,"Политология,Книги по экономике,Газеты",Authors_Unknown,2019
59595,321616,МК Московский Комсомолец 292-2019,"Политология,Книги по экономике,Газеты",Authors_Unknown,2019
59596,125582,Известия 248-249-2019,"Политология,Общая история,Газеты",Authors_Unknown,2019
59597,33188,Men's Health 01-2020,Журнальные издания,Authors_Unknown,2019


In [53]:
users_df_first['user_id'].unique().sort()

In [54]:
aaa = users_df_first['user_id'].unique()


aaa.sort()

In [55]:
aaa

array([     1,      2,      3, ..., 159609, 159610, 159611])